<a href="https://colab.research.google.com/github/kolatejaswi/Tejaswi-Introduction-to-Gen-AI-and-simple-LLM-inference-on-CPU-finetuning-to-create-custom-chatbot/blob/main/Fine%20tuning%20model%20to%20create%20custom%20chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [ ]:
import pandas as pd
df=pd.read_csv('/content/gdrive/My Drive/winking_face.csv',encoding='latin-1')
df.head()

,Text
0,@Holy_Trinity_AV The Prince and Wales and Prin...
1,@Answer4today @BeutelDory @krassenstein Iâm ...
2,@ClayMoore1966 @Jerrypatriot @JoeinWashDC @Fox...
3,@jackobt Wow. Worth the drive ð
4,"that's me going everywhere, because someone ha..."


In [ ]:
print(df.columns)

Index(['Text'], dtype='object')


In [ ]:
import pandas as pd
# Preprocess the data to combine tweets and emojis into a single text field
df['text'] = df['Text']

# Select only the text column for training
texts = df['text'].tolist()

In [ ]:
!pip install transformers datasets torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [ ]:
split_index = int(0.8 * len(df))
df_train = df[:split_index]
df_validation = df[split_index:]

# Display the DataFrames
print(df_train.head())
print(df_validation.head())


                                                Text  \
0  @Holy_Trinity_AV The Prince and Wales and Prin...   
1  @Answer4today @BeutelDory @krassenstein Iâm ...   
2  @ClayMoore1966 @Jerrypatriot @JoeinWashDC @Fox...   
3                 @jackobt Wow. Worth the drive ð   
4  that's me going everywhere, because someone ha...   

                                                text  
0  @Holy_Trinity_AV The Prince and Wales and Prin...  
1  @Answer4today @BeutelDory @krassenstein Iâm ...  
2  @ClayMoore1966 @Jerrypatriot @JoeinWashDC @Fox...  
3                 @jackobt Wow. Worth the drive ð  
4  that's me going everywhere, because someone ha...  
                                                    Text  \
16000  Copy of Charity evening Psychic Mediumship Dem...   
16001  @attorneyjeremy1 Jeremy has apparently discove...   
16002  Yeah itâs just God letting me know whatâs ...   
16003  @SKouklaaa07 they are lucky to have JK ð ah...   
16004       And so is your 9 neve

In [ ]:
!pip install emoji==2.2.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234912 sha256=c3a8393e2d0e8d0392c1617eeab0088bca54c12bcbac8bb8f058e4233d8387d2
  Stored in directory: /root/.cache/pip/wheels/02/3d/88/51a592b9ad17e7899126563698b4e3961983ebe85747228ba6
Successfully built emoji


In [ ]:
import emoji

def contains_emoji(text):
    for character in text:
        if character in emoji.EMOJI_DATA:
            return True
    return False

# Check if any text in the 'Text' column contains emojis
has_emojis = df['Text'].apply(contains_emoji).any()

if has_emojis:
    print("The dataset contains emojis! 🎉")
else:
    print("The dataset does not contain emojis. 😔")

The dataset contains emojis! 🎉


In [ ]:
from transformers import GPT2Tokenizer

# Initialize the GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
# Add a padding token to the tokenizer
tokenizer.pad_token = tokenizer.eos_token # Use the end-of-sequence token as the padding token

# Tokenize the training and validation datasets
def tokenize_function(examples):
    return tokenizer(examples["Text"], padding="max_length", truncation=True)

df_train["tokens"] = df_train["Text"].apply(lambda x: tokenizer(x, padding="max_length", truncation=True))
df_validation["tokens"] = df_validation["Text"].apply(lambda x: tokenizer(x, padding="max_length", truncation=True))

# Display tokenized data
print(df_train.head())
print(df_validation.head())


<ipython-input-20-79dc6918879b>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train["tokens"] = df_train["Text"].apply(lambda x: tokenizer(x, padding="max_length", truncation=True))


                                                Text  \
0  @Holy_Trinity_AV The Prince and Wales and Prin...   
1  @Answer4today @BeutelDory @krassenstein Iâm ...   
2  @ClayMoore1966 @Jerrypatriot @JoeinWashDC @Fox...   
3                 @jackobt Wow. Worth the drive ð   
4  that's me going everywhere, because someone ha...   

                                                text  \
0  @Holy_Trinity_AV The Prince and Wales and Prin...   
1  @Answer4today @BeutelDory @krassenstein Iâm ...   
2  @ClayMoore1966 @Jerrypatriot @JoeinWashDC @Fox...   
3                 @jackobt Wow. Worth the drive ð   
4  that's me going everywhere, because someone ha...   

                        tokens  
0  [input_ids, attention_mask]  
1  [input_ids, attention_mask]  
2  [input_ids, attention_mask]  
3  [input_ids, attention_mask]  
4  [input_ids, attention_mask]  
                                                    Text  \
16000  Copy of Charity evening Psychic Mediumship Dem...   
16001  @

<ipython-input-20-79dc6918879b>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_validation["tokens"] = df_validation["Text"].apply(lambda x: tokenizer(x, padding="max_length", truncation=True))


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

class ConversationDataset(Dataset):
    def __init__(self, df):
        self.input_ids = df["tokens"].apply(lambda x: x["input_ids"]).tolist()
        self.attention_masks = df["tokens"].apply(lambda x: x["attention_mask"]).tolist()

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            "input_ids": torch.tensor(self.input_ids[idx]),
            "attention_mask": torch.tensor(self.attention_masks[idx])
        }

# Create DataLoader for the training and validation datasets
train_dataset = ConversationDataset(df_train)
validation_dataset = ConversationDataset(df_validation)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=8)


In [ ]:
from transformers import GPT2LMHeadModel
# Load the GPT-2 model
model = GPT2LMHeadModel.from_pretrained('gpt2')


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
def generate_response(prompt):
    inputs = tokenizer(prompt, return_tensors='pt')
    outputs = model.generate(inputs.input_ids, max_length=50, num_return_sequences=1)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response
prompt = "I love emojis! ✨"
input_ids = tokenizer.encode(prompt, return_tensors="pt")
output = model.generate(input_ids, max_length=50, num_return_sequences=1, do_sample=True, temperature=0.7)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)




The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


I love emojis! ✨️🙏🏼✨️🙏🏼✨️ 🙏🏼✨️🙏🏼✨️🙏�


In [ ]:
prompt = "I'm so excited about this project! 🎉 How can I get started?"
input_ids = tokenizer.encode(prompt, return_tensors="pt")
output = model.generate(input_ids, max_length=50, num_return_sequences=1, do_sample=True, temperature=0.7)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


I'm so excited about this project! 🎉 How can I get started? 💯 🎄 Make a donation to my campaign and help me continue to be the best person I can be! 💅 💰 I will
